In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [2]:
class Robot:
    def __init__(self,pos:np.ndarray,antennas:list):
        self.antennas = antennas
        self.position = pos
        self.num_of_antennas = len(antennas)
    def update_pos(self,pos):
        self.position = pos

In [3]:
class Tag:
    def __init__(self,pos:np.ndarray):
        self.position = pos

In [4]:
class Antenna:
    def __init__(self,z:float,G_dBm:float,theta_vector:np.ndarray,phi_vector:np.ndarray):
        self.z = z
        self.G_dBm = G_dBm
        self.theta_vector = theta_vector
        self.phi_vector = phi_vector
    
    def get_gain(self,theta_rad:float,phi_rad:float):
        theta_deg: int = round(theta_rad*180/np.pi)%360 #we don't really need the modulo here, i think but just to be safe
        phi_deg: int = round(phi_rad * 180/np.pi)%360
        return self.theta_vector[theta_deg] + self.phi_vector[phi_deg] + self.G_dBm


In [ ]:
#We assume the RFID tag frequency is 866MHz
freq = 866e6
#Speed of light
c = 299792458.0

lambda_signal = c/freq


Pt_dBm = 30

G_R_dBm = 1

#antenna initialization
pattern = pd.read_excel("pattern.xlsx")
gain_theta = pattern["Gain_t(dB)"].to_numpy()
gain_phi = pattern["Gain_p(dB)"].to_numpy()
Antenna1 = Antenna(0.2,4,gain_theta,gain_phi)
#Antenna2 = Antenna(0.4,4,gain_theta,gain_phi)

#Points of the movement (for the moment it is straight)
p1 = np.array([0,0,0])
p2 = np.array([4,0,0])
#Initialization of robot with antennas
Robot_inst = Robot(p1,[Antenna1])
#p3 serves as a visual guide to know where the movement starts
kappa = 0.1
p3 = (1-kappa)*p1+kappa*p2


data_list = []  #list containing all the saved data
#tag position params
x_tag = 2 
z_tag = 0.1
y_steps = np.linspace(0.1,3,1000) #the last number is the number of samples for each epoch
for y_tag in y_steps:
    #Tag initialization
    tagp = np.array([x_tag,y_tag,z_tag])
    tag1 = Tag(tagp)
    #do this for all antennas
    for antenna_num,antenna in enumerate(Robot_inst.antennas):
        phases = []
        robot_pos = []
        rho = 0   #rho is the position of the x axis
        mu,sigma = 0.0038,0.001834  #mu and sigma for the normal distribution of the sampling 
        while(rho<4):
            rho = rho + np.random.normal(mu,sigma)
            #there will be a problem with z if p1 and p2 do not have z = 0
            new_pos = p1 + rho*(p2-p1)/np.linalg.norm(p2-p1)
            robot_pos.append(new_pos)
            Robot_inst.update_pos(new_pos)
            r = np.linalg.norm(tag1.position-Robot_inst.position)
            #Find the phi,theta of the antenna pattern and move it from -pi,pi to 0,2pi
            if(tag1.position[0] == 0):
                theta_tag = np.pi/2
            else:
                theta_tag = np.atan(tag1.position[1]/tag1.position[0])
            theta_tag = (theta_tag + 2*np.pi)%np.pi
            if(tag1.position[2] == antenna.z):
                phi_tag = 0
            else:
                phi_tag = np.atan(np.sqrt(pow(tag1.position[0],2)+pow(tag1.position[1],2))/(tag1.position[2]-antenna.z))
            phi_tag = (phi_tag + 2*np.pi)%np.pi
            #we calculate the Friis equation
            Pr = antenna.get_gain(theta_tag,phi_tag) + G_R_dBm + Pt_dBm + 20*np.log10(lambda_signal/(4*np.pi*r))
            phases.append( ((2*np.pi)/lambda_signal)*2*r + np.random.normal(0,0.09)) #mu = 0 and sigma = 5deg ~= 0.09 rad
        data_list.append({
            "x_tag": x_tag,
            "y_tag": y_tag,
            "z_tag": z_tag,
            "Antenna": antenna_num,
            "robot_pos" : np.array(robot_pos),
            "Phases": np.array(phases)
        })
    
    ##VISUALIZATION

    # fig = plt.figure()
    # plt.scatter(np.array([p1[0],p3[0],p2[0],tagp[0]]),np.array([p1[1],p3[1],p2[1],tagp[1]]))
    # plt.show()
    
    # fig = plt.figure()
    # x = np.arange(0,len(phases))
    # plt.scatter(x,phases)
    # plt.show()

    # fig = plt.figure()
    # x = np.arange(0,len(phases))
    # plt.scatter(x,np.array(phases)%(2*np.pi),linewidths=0.2)
    # plt.show()

    # fig = plt.figure()
    # plt.plot(gain_theta)
    # plt.plot(gain_phi)
    # plt.show()

# Convert the list to a DataFrame
df = pd.DataFrame(data_list)

# Save to an Excel file
df.to_pickle("Sim_Data_Noisy.pkl")





The below code has improved on the cell after this

In [6]:
Data = pd.read_pickle("Sim_Data_Noisy.pkl")

y = Data["y_tag"].to_numpy()
phases = Data["Phases"].to_numpy()
original_pos = Data["robot_pos"]
num_of_samples = 1000
num_of_ys = len(phases)
data_list = []

for i in range(num_of_ys):
    phase = phases[i]
    org_pos = original_pos[i]
    org_x = org_pos[:,0]

    x_new = np.linspace(org_x[0],org_x[-1],num_of_samples)
    linear_interp_phase = interp1d(org_x,phase,kind='linear',fill_value='extrapolate')
    phase_new = linear_interp_phase(x_new)
    data_list.append({
        "x_tag": Data['x_tag'][i],
        "y_tag": Data['y_tag'][i],
        "z_tag": Data['z_tag'][i],
        "Antenna": Data['Antenna'][i],
        "robot_pos" : np.array([x_new, np.repeat(org_pos[0,1],len(x_new)), np.repeat(org_pos[0,2],len(x_new))]), # Different x's but same y and z's as read
        "Phases": np.array(phase_new)
    })
    
    ##VISUALIZATION

    # plt.figure(figsize=(8, 4))
    # plt.plot(org_x, phase, '-', label='Original Data')
    # #plt.plot(x_new, phase_new, '-', label='Interpolated Data')
    # plt.xlabel('x')
    # plt.ylabel('f(x)')
    # plt.title('Linear Interpolation of f(x)')
    # plt.legend()
    # plt.show()

df = pd.DataFrame(data_list)


df.to_pickle("Sim_Data_Noisy_Interp.pkl")